In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 3

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed) 
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",
            'fairness_agg': 'mean',

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 128,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 5,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha': 0.002,
            'lambda': 1.0,
            'lambda_old': 0.0,
              }
    


#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    if params['lambda'] != 0:
        trial_id+=f"_lmbd_{params['lambda']}_lmbdold_{params['lambda_old']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=BiasedMNIST/seed=3_epoch=1_lr=0.001_tau=5_alpha=0.002_lmbd_1.0_lmbdold_0.0


In [2]:
"MNIST" in params['dataset']

True

In [3]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'])

    n_feature = 3*28*28

[0 1 2 3 4 5 6 7 8 9]


In [4]:
from torchvision.transforms.functional import to_pil_image
from cl_gym.benchmarks.transforms import MNIST_MEAN, MNIST_STD
COLOR_MAP = {
    0: (1, 0, 0),
    1: (0, 1, 0),
    2: (1, 1, 0),
    3: (0, 0, 1),
    4: (1, 0.65, 0),
    5: (0.5, 0, 0.5),
    6: (0, 1, 1),
    7: (1, 0.75, 0.8),
    8: (0.8, 1, 0),
    9: (.588, .294, 0.)
}

r = (1 - 0.1913)
color_values = np.array(list(COLOR_MAP.values()))
m_rgb = color_values.mean(axis=0)
std_rgb = color_values.std(axis=0)

bmnist_mean = [r*m + MNIST_MEAN[0] for m in m_rgb]
bmnist_std = [(r*s**2+(1-r)*MNIST_STD[0]**2+r*(1-r)*(bmnist_mean[i] - m_rgb[i])**2)**0.5 for i, s in enumerate(std_rgb)]

unnormalize = torchvision.transforms.Normalize([-m/s for m, s in zip(bmnist_mean, bmnist_std)], [1/s for s in bmnist_std])
sample_idx = 95
to_pil_image(unnormalize(benchmark.trains[2][sample_idx][0]), mode="RGB")

In [5]:
target_label = 0
incremental_step = 1
# cat_img = torch.cat([img for img in benchmark.trains[incremental_step].inputs[benchmark.trains[incremental_step].targets == target_label][20:30]], dim=2)
cat_img = torch.cat([img for img, target, *_ in benchmark.tests[incremental_step]][20:30], dim=2)
to_pil_image(unnormalize(cat_img), mode="RGB")

In [6]:
from trainers import FairContinualTrainer
from trainers.fair_trainer import FairContinualTrainer2
from metrics import FairMetricCollector
from algorithms.sensitive import Heuristic3

backbone = cl.backbones.MLP2Layers(input_dim=n_feature, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic3(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

trainer = FairContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
benchmark.trains[1][0][0].shape

torch.Size([3, 28, 28])

In [8]:
if params['fairness_agg'] == "mean":
    agg = np.mean
elif params['fairness_agg'] == "max":
    agg = np.max
else:
    raise NotImplementedError

metric_manager_callback.meters['multiclass_eo'].agg = agg
metric_manager_callback.meters['multiclass_eo'].agg


<function mean at 0x7fe86428a0b0>

In [9]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------


[1] Eval metrics for task 1 >> {'accuracy': 80.73811022206239, 'loss': 0.0024567958874623544, 'multiclass_eo': [0.2655601659751037, 0.5088028169014085], 'accuracy_s0': 100.0, 'accuracy_s1': 61.281850856174394, 'classwise_accuracy': {0: array([852, 980]), 1: array([ 846, 1135])}}
training_task_end
---------------------------- Task 2 -----------------------
losses=tensor([[6.3783e-03, 9.6620e-04, 4.0494e+00, 2.8714e+00, 1.0421e+00, 1.2290e+00,
         4.8000e+00, 5.2367e+00]])
A_np.shape=(8, 10000)
b_np.shape=(8,)
### Scipy minsum LP solver ###
Elapsed time:0.118
Loss difference:[0.35890835 0.26762537 0.23151184 0.6440317  0.         0.
 0.         0.        ]
len(updated_seq_indices)=4355
[2] Eval metrics for task 1 >> {'accuracy': 98.42308729659266, 'loss': 0.0003449792628028996, 'multiclass_eo': [0.014084507042253502, 0.045709810195137424], 'accuracy_s0': 99.8995983935743, 'accuracy_s1': 96.90988253170475, 'classwise_accuracy': {1: array([1127, 1135]), 0: array([956, 980])}}
[2] Eval

In [10]:
metric_manager_callback.meters['accuracy'].get_data()

array([[80.738,  0.   ,  0.   ,  0.   ,  0.   ],
       [98.423, 57.598,  0.   ,  0.   ,  0.   ],
       [95.399, 90.372, 61.948,  0.   ,  0.   ],
       [90.316, 83.861, 84.438, 55.603,  0.   ],
       [92.512, 79.306, 84.082, 90.176, 63.606]])

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['accuracy'].compute_overall()]

[94.781, 90.57, 89.316, 87.204, 84.87]

In [ ]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

89.3481835003114

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['multiclass_eo'].compute_overall()]

[0.105, 0.146, 0.177, 0.165, 0.197]

In [ ]:
np.mean(metric_manager_callback.meters['multiclass_eo'].compute_overall())

0.15817295661807243

In [ ]:
task_id = 2

print(f"{task_id=}")
print(f"sensitive samples / all samples = {(benchmark.trains[task_id].sensitives != benchmark.trains[task_id].targets).sum().item()} / {benchmark.trains[task_id].sensitives.shape[0]}")

updated_seq_indices = benchmark.seq_indices_train[task_id]
print(f"sensitive samples / selected samples = {(benchmark.trains[task_id].sensitives[updated_seq_indices] != benchmark.trains[task_id].targets[updated_seq_indices]).sum().item()} / {len(updated_seq_indices)}")


task_id=2
sensitive samples / all samples = 613 / 12089
sensitive samples / selected samples = 415 / 704


In [ ]:
step_class = 2
for i in range(2, 6):
    incremental_step = i
    print(f"{incremental_step=}")
    one_idx = benchmark.trains[incremental_step].sample_weight > 0.9

    print(f"{2*i-2} : {(benchmark.trains[incremental_step].targets == (2*i-2)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-2)).sum().item()}")
    print(f"{2*i-1} : {(benchmark.trains[incremental_step].targets == (2*i-1)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-1)).sum().item()}")


incremental_step=2
2 : 5958 --> 1238
3 : 6131 --> 1554
incremental_step=3
4 : 5842 --> 1410
5 : 5421 --> 1921
incremental_step=4
6 : 5918 --> 1653
7 : 6265 --> 1590
incremental_step=5
8 : 5851 --> 1666
9 : 5949 --> 2019


In [ ]:
for eos in metric_manager_callback.meters['multiclass_eo'].get_data():
    for eo in eos:
        print(round(eo, 3),end=" ")
    print()

0.046 0.164 
0.108 0.065 0.062 0.35 
0.113 0.083 0.212 0.104 0.152 0.402 
0.13 0.117 0.208 0.145 0.202 0.214 0.114 0.19 
0.186 0.123 0.125 0.268 0.281 0.236 0.212 0.134 0.15 0.259 


In [ ]:
metric_manager_callback.meters['classwise_accuracy'].get_data()

[{0: array([958, 980]), 1: array([1042, 1135])},
 {0: array([914, 980]),
  1: array([1092, 1135]),
  3: array([ 980, 1010]),
  2: array([ 782, 1032])},
 {0: array([906, 980]),
  1: array([1080, 1135]),
  3: array([ 873, 1010]),
  2: array([ 945, 1032]),
  4: array([906, 982]),
  5: array([696, 892])},
 {1: array([1043, 1135]),
  0: array([886, 980]),
  3: array([ 848, 1010]),
  2: array([ 855, 1032]),
  5: array([752, 892]),
  4: array([843, 982]),
  7: array([ 943, 1028]),
  6: array([830, 958])},
 {0: array([865, 980]),
  1: array([1037, 1135]),
  2: array([ 847, 1032]),
  3: array([ 723, 1010]),
  4: array([791, 982]),
  5: array([743, 892]),
  7: array([ 891, 1028]),
  6: array([863, 958]),
  8: array([885, 974]),
  9: array([ 847, 1009])}]